In [1]:
from transformers import DistilBertTokenizerFast
from datasets import load_dataset

# Load dataset
dataset = load_dataset('glue', 'mrpc')

# Initialize tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2', 'idx'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format('torch')


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer

# Load pre-trained model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)


2024-10-22 08:18:11.239209: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 08:18:11.252252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 08:18:11.266159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 08:18:11.270218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 08:18:11.281063: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from codecarbon import EmissionsTracker
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize the emissions tracker
tracker = EmissionsTracker()
tracker.start()

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator = data_collator,
)

# Start training
trainer.train()

# Stop the tracker and print emissions
emissions: float = tracker.stop()
print(f"Emissions: {emissions} kg CO₂")


[codecarbon ERROR @ 08:18:13] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.
[codecarbon WARNING @ 08:18:13] Another instance of codecarbon is already running. Exiting.
[codecarbon ERROR @ 08:18:13] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.
[codecarbon WARNING @ 08:18:14] Another instance of codecarbon is already running. Exiting.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.396495


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
[codecarbon WARNING @ 08:18:26] Another instance of codecarbon is already running. Exiting.
[codecarbon WARNING @ 08:18:26] Another instance of codecarbon is already running. Exiting.


Emissions: None kg CO₂


In [ ]:
#dotprod 128: 2000j 80s 64: 1400 56s 256: 3500j 170s 
#inhibitor 128: 3700j 170s 1860j 80s 256: 10700 570s

#int8 inhibitor :32: 460J 17.55 64: 1070J 40S 128: 2500j 110s
#int8 dotprod :32: 370j 16s 64: 480J 17s 128: 861J 27s